### Imports

In [3]:
import pandas as pd
from pycaret.regression import *
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import pickle
from datetime import timedelta
import talib
import numpy as np

In [4]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Dataset

In [5]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

columns_to_model = [
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    # 'SMA_20',
    # 'EMA_20',
    # 'Upper_Band',
    # 'Middle_Band',
    # 'Lower_Band',
    # 'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

In [6]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])

display(dataset.head())

,Open_time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Sentimiento,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Sentimiento_coin,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Sentimiento_referentes,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
0,2021-09-27,28.84,30.26,27.50,27.71,11094418.10,320340017.82,449178.00,43160.90,44350.00,42098.00,42147.35,39776.84,1728912299.80,1001487.00,3059.36,3164.94,2922.71,2925.59,510130.73,1564096695.37,1043885.00,344.00,357.60,335.00,335.50,956544.07,332372476.51,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.02,0.96,21.20,24.79,33.78,-86.30,4.00,29.23,0.00,0.00,0.00,pos,139,61.00,28.00,50.00,17.83,pos,135,71.00,26.00,38.00,30.62,neu,1.00,0.00,0.00,1.00,0.00,81.00,525.00,364.00,270000.00,Lateral
1,2021-09-28,27.72,28.38,26.14,26.31,9784154.53,267560973.69,362304.00,42147.35,42787.38,40888.00,41026.54,43372.26,1816305387.84,1045389.00,2924.76,2970.82,2789.00,2804.91,511325.46,1477983245.67,928494.00,335.50,344.60,330.00,333.00,922077.23,310828343.70,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,-0.33,0.70,21.98,16.10,24.47,-109.46,3.88,19.19,4.00,2.00,5273.00,pos,93,40.00,21.00,32.00,10.18,pos,122,57.00,23.00,42.00,19.89,neu,2.00,1.00,1.00,0.00,0.02,87.00,472.00,331.00,204000.00,Bajista
2,2021-09-29,26.31,28.59,26.11,27.28,10207287.33,280535376.38,376232.00,41025.01,42590.00,40753.88,41524.28,33511.53,1399625476.06,884909.00,2805.15,2950.00,2782.00,2850.45,411305.09,1180681056.16,748804.00,332.90,375.40,331.20,367.70,1696420.04,612547661.88,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,-0.52,0.46,22.61,12.88,17.92,-92.93,3.78,23.82,22.00,40.00,54144.00,pos,112,61.00,19.00,32.00,23.13,pos,145,71.00,20.00,54.00,30.30,neu,0.00,0.00,0.00,0.00,0.00,64.00,594.00,495.00,216000.00,Alcista
3,2021-09-30,27.28,28.99,27.13,28.62,8201551.44,230622998.43,339737.00,41524.29,44141.37,41410.17,43824.10,46381.23,2005486003.56,1197815.00,2850.45,3048.42,2837.19,3000.61,506896.76,1514341937.31,992243.00,367.70,388.80,366.40,387.50,1163674.21,439628574.42,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,-0.56,0.26,22.99,16.42,15.13,-60.03,3.64,28.34,15.00,23.00,39220.00,pos,116,46.00,28.00,42.00,7.68,pos,147,73.00,26.00,48.00,30.90,pos,2.00,1.00,0.00,1.00,0.75,77.00,419.00,464.00,202000.00,Alcista
4,2021-10-01,28.61,32.33,28.50,31.94,13176348.21,402700970.65,735059.00,43820.01,48495.00,43283.03,48141.61,66244.87,3076094886.59,1771237.00,3000.62,3330.00,2969.07,3309.91,648714.62,2063112280.67,1446386.00,387.50,423.40,381.50,421.50,1440336.04,584311192.72,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,-0.31,0.14,21.86,34.90,21.40,34.17,3.65,35.02,24.00,35.00,63183.00,pos,171,72.00,21.00,78.00,30.72,pos,141,70.00,18.00,53.00,30.43,pos,1.00,1.00,0.00,0.00,0.82,71.00,477.00,664.00,492000.00,Alcista


In [7]:
dates = dataset['Open_time'][:-5]

dataset = dataset.round(2)

feature_dataset = dataset[columns]
# feature_dataset['Open_time'] = dataset['Open_time']
# feature_dataset.set_index('Open_time', inplace=True)

validation = feature_dataset[-5:]
feature_dataset = feature_dataset[:-5]

n_days_to_predict = 5

In [8]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,2021-09-27,28.84,30.26,27.50,27.71,449178.00,42147.35,39776.84,1001487.00,2925.59,510130.73,1043885.00,335.50,956544.07,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.00,0.00,0.00,139,135,1.00,81.00,525.00,364.00,270000.00
1,2021-09-28,27.72,28.38,26.14,26.31,362304.00,41026.54,43372.26,1045389.00,2804.91,511325.46,928494.00,333.00,922077.23,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,4.00,2.00,5273.00,93,122,2.00,87.00,472.00,331.00,204000.00
2,2021-09-29,26.31,28.59,26.11,27.28,376232.00,41524.28,33511.53,884909.00,2850.45,411305.09,748804.00,367.70,1696420.04,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,22.00,40.00,54144.00,112,145,0.00,64.00,594.00,495.00,216000.00
3,2021-09-30,27.28,28.99,27.13,28.62,339737.00,43824.10,46381.23,1197815.00,3000.61,506896.76,992243.00,387.50,1163674.21,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,15.00,23.00,39220.00,116,147,2.00,77.00,419.00,464.00,202000.00
4,2021-10-01,28.61,32.33,28.50,31.94,735059.00,48141.61,66244.87,1771237.00,3309.91,648714.62,1446386.00,421.50,1440336.04,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,24.00,35.00,63183.00,171,141,1.00,71.00,477.00,664.00,492000.00


(903, 31)


,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,2024-03-18,10.08,10.46,9.60,9.90,245319.00,67609.99,55691.08,2464515.00,3520.46,570901.29,1906387.00,555.40,2284301.81,994512.00,10.06,9.95,11.86,10.06,8.26,52.48,34.00,43.00,84706.00,696,471,0.00,43.00,343.00,228.00,154000.00
904,2024-03-19,9.90,9.99,8.60,8.77,341363.00,61937.40,101005.32,3593832.00,3158.64,1049629.69,2647385.00,507.70,2551361.51,1213572.00,10.08,9.84,11.81,10.08,8.35,42.93,120.00,126.00,135180.00,961,509,1.00,56.00,534.00,433.00,221000.00
905,2024-03-20,8.77,9.57,8.49,9.48,267797.00,67840.51,90420.59,3549793.00,3516.53,1207322.82,2987953.00,556.80,1425296.58,809335.00,10.14,9.80,11.68,10.14,8.60,49.21,185.00,117.00,112997.00,866,555,1.00,40.00,473.00,386.00,171000.00
906,2024-03-21,9.48,9.58,9.07,9.18,156774.00,65501.27,53357.48,2388390.00,3492.85,602755.21,1791989.00,553.80,953921.37,563996.00,10.17,9.74,11.63,10.17,8.71,46.85,64.00,81.00,66543.00,692,533,0.00,24.00,350.00,290.00,101000.00
907,2024-03-22,9.18,9.37,8.69,8.94,147578.00,63796.64,51482.38,2492881.00,3336.35,558848.89,1747756.00,553.80,1181298.51,712381.00,10.14,9.67,11.67,10.14,8.62,45.00,57.00,66.00,68616.00,681,546,0.00,41.00,252.00,206.00,92000.00


(5, 31)

### Model

#### Generacion de los modelos

In [7]:
# train_size = 0.8
# train_data, test_data = train_test_split(feature_dataset, train_size=train_size, shuffle=False)

In [14]:
h2o.init()

selected_models = {}

for column in columns_to_model:
    
    # A cada registro, le asigno como target el valor de Column del dia siguiente,
    # de esta forma, los N modelos son entrenados para predecir el valor de los N
    # atributos en un dia siguiente.
    training_data = feature_dataset
    training_data['Next_Day_Target'] = feature_dataset[column].shift(-1)
    training_data = training_data[:-1] #Tengo que sacar el ultimo registro porque no tendra valor del target
    ##
    h2o_df = h2o.H2OFrame(training_data.drop(columns=["Open_time"]))

    aml = H2OAutoML(max_models=25, seed=1234) #TODO: SACAR EL max_models
    aml.train(y='Next_Day_Target', training_frame=h2o_df)

    leaderboard = aml.leaderboard
    print(leaderboard)

    top_models = leaderboard.head(3)["model_id"].as_data_frame()["model_id"].tolist()

    best_model_id = top_models[0]
    best_model = h2o.get_model(best_model_id)
    
    selected_models[column] = best_model

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 days 2 hours 33 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_mmarchetta_dfdxaj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.018 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [9]:
predictions_dict = {}

for column, model in selected_models.items():
    predictions = model.predict(h2o.H2OFrame(validation.drop(columns=["Open_time"])))
    predictions_dict[column] = predictions

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [10]:
for column, predictions in predictions_dict.items():
    print(f"Predicciones para los próximos días de {column}:")
    print(predictions)

Predicciones para los próximos días de Open:
  predict
  9.9575
  8.94215
  9.32918
  9.28929
  9.01623
[5 rows x 1 column]

Predicciones para los próximos días de High:
  predict
 10.0663
  9.14965
  9.32176
  9.01123
  8.79655
[5 rows x 1 column]

Predicciones para los próximos días de Low:
  predict
  9.16624
  7.79157
  8.32764
  8.50675
  8.22469
[5 rows x 1 column]

Predicciones para los próximos días de Close:
  predict
 10.0949
  9.37216
  9.37297
  9.63515
  9.27944
[5 rows x 1 column]

Predicciones para los próximos días de Number of trades:
  predict
   202467
   244845
   263979
   222654
   207142
[5 rows x 1 column]

Predicciones para los próximos días de Close_BTCUSDT:
  predict
  68772.5
  60050.3
  66507
  66238.6
  63757.2
[5 rows x 1 column]

Predicciones para los próximos días de Volume_BTCUSDT:
  predict
  41820.5
  57117.4
  52410.8
  40583.1
  40219.9
[5 rows x 1 column]

Predicciones para los próximos días de Number_of_trades_BTCUSDT:
    predict
2.24456e+06
2.4

#### Guardo el mejor modelo para cada columna

In [11]:
for column, model in selected_models.items():
    file_name = f"modelo_{column}.h2o"
    h2o.save_model(model, 'h2o_models', filename=file_name)

#### Vuelvo a armar el mejor modelo para cada columna

In [10]:
# h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 days 2 hours 32 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_mmarchetta_dfdxaj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.018 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [11]:
loaded_models = {}
for column in columns_to_model:
    model_path = f"modelo_{column}.h2o"
    loaded_models[column] = h2o.load_model(f"h2o_models/{model_path}")

#### Prediccion 1 dia en el futuro

In [12]:
def hacer_predicciones_para_dia_siguiente(trained_models, last_day_data):
    """
    Predice los valores para el día siguiente utilizando los modelos entrenados y el último registro de datos.

    Args:
    - trained_models: Un diccionario donde las claves son las características y los valores son los modelos entrenados.
    - last_day_data: Pandas Series que representa el último registro de datos con todas las características excepto la variable objetivo.

    Returns:
    - Pandas DataFrame con un solo registro que contiene las predicciones para el día siguiente.
    """
    # Crear un diccionario para almacenar las predicciones
    predictions = {}

    # Hacer predicciones para cada característica utilizando los modelos entrenados
    for feature, model in trained_models.items():
        # Preparar los datos para la predicción (convertir a DataFrame de H2O)
        h2o_last_day_data = h2o.H2OFrame(last_day_data)
        
        # Realizar la predicción para la característica actual
        prediction = model.predict(h2o_last_day_data).as_data_frame().iloc[0, 0]
        
        # Almacenar la predicción en el diccionario
        predictions[feature] = prediction

    # Convertir el diccionario de predicciones en un DataFrame de pandas
    predictions_df = pd.DataFrame(predictions, index=[0])

    return predictions_df

In [15]:
predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])

# Muestra el DataFrame con las predicciones para el día siguiente
display(predictions_next_day)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,10.02,10.04,9.25,10.25,234059.73,68244.09,45381.60,2286466.77,3682.46,610589.41,1803467.86,537.93,1432914.75,751748.41,56.48,38.32,70371.80,595.77,299.63,0.85,41.51,400.59,294.12,154147.20


#### Prediccion de los N dias en el futuro

In [16]:
def calcular_indicadores_tecnicos(historico_precio):
    close_prices = np.array(historico_precio['Close'], dtype=np.double)
    # high_prices = np.array(historico_precio['High'], dtype=np.double)
    # low_prices = np.array(historico_precio['Low'], dtype=np.double)

    # Media Móvil Simple (SMA)
    sma_20 = talib.SMA(close_prices, timeperiod=20)

    # Media Móvil Exponencial (EMA)
    ema_20 = talib.EMA(close_prices, timeperiod=20)

    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=20, nbdevup=2, nbdevdn=2)

    # Índice de Fuerza Relativa (RSI)
    rsi = talib.RSI(close_prices, timeperiod=14)

    # Crear un DataFrame para almacenar los indicadores técnicos
    indicadores = pd.DataFrame({
        'Open_time': historico_precio['Open_time'],
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi
    }, index=historico_precio.index)

    return indicadores

In [17]:
cantidad_dias_futuro = 5

for i in range(cantidad_dias_futuro):
    predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])
    
    ultima_fecha = feature_dataset["Open_time"].iloc[-1]
    nueva_fecha = ultima_fecha + timedelta(days=1)
    
    nuevo_registro = pd.DataFrame({"Open_time": [nueva_fecha]})

    for column, prediction in predictions_next_day.items():
        nuevo_registro[column] = prediction
        
    feature_dataset = pd.concat([feature_dataset, nuevo_registro], ignore_index=True)

    nuevos_indicadores = calcular_indicadores_tecnicos(feature_dataset)
    
    for column in nuevos_indicadores.columns:
        feature_dataset.loc[feature_dataset.index[-1], column] = nuevos_indicadores.loc[nuevos_indicadores.index[-1], column]
    
    # display(nuevos_indicadores)mm
    

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |██████████████████████████████████████████████████

In [21]:
display(feature_dataset.tail(10))

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Next_Day_Target
898,2024-03-13,10.74,11.46,10.64,11.37,276468.00,73072.41,52659.71,2501197.00,4004.79,482305.78,1536498.00,630.50,2526002.56,1265237.00,9.45,9.70,11.82,9.45,7.07,73.43,64.00,81.00,92576.00,275.00,205.00,1.00,73.00,578.00,553.00,164000.00,327000.00
899,2024-03-14,11.37,11.89,10.68,11.56,536988.00,71388.94,71757.63,2994869.00,3881.70,648237.52,1919963.00,603.20,2119540.30,1038297.00,9.65,9.88,12.03,9.65,7.27,74.51,102.00,133.00,145727.00,211.00,181.00,4.00,29.00,754.00,677.00,327000.00,360000.00
900,2024-03-15,11.56,11.71,9.98,10.81,557152.00,69499.85,103334.04,3904445.00,3742.19,947537.41,2487337.00,632.70,3066312.79,1365283.00,9.80,9.97,12.07,9.80,7.52,63.27,88.00,83.00,147460.00,238.00,106.00,0.00,25.00,493.00,430.00,360000.00,209000.00
901,2024-03-16,10.81,10.90,9.50,9.68,330505.00,65300.63,55926.95,2729019.00,3523.09,548288.16,1798939.00,576.40,1811838.04,1025452.00,9.89,9.94,11.99,9.89,7.78,50.82,30.00,49.00,88095.00,670.00,471.00,0.00,20.00,513.00,403.00,209000.00,150000.00
902,2024-03-17,9.68,10.25,9.19,10.08,229683.00,68393.48,49742.22,2449156.00,3644.71,517790.99,1721355.00,571.70,1712920.34,802297.00,9.98,9.95,11.93,9.98,8.04,54.27,36.00,48.00,71390.00,693.00,413.00,0.00,21.00,295.00,277.00,150000.00,NaN
903,2024-03-18,10.02,10.04,9.25,10.25,234059.73,68244.09,45381.60,2286466.77,3682.46,610589.41,1803467.86,537.93,1432914.75,751748.41,10.08,9.98,11.88,10.08,8.28,55.68,56.48,38.32,70371.80,595.77,299.63,0.85,41.51,400.59,294.12,154147.20,NaN
904,2024-03-19,10.01,10.20,9.33,10.47,239360.17,68107.54,43405.81,2027345.35,3707.93,640074.93,1763927.76,530.86,1307452.52,723372.56,10.18,10.03,11.80,10.18,8.56,57.49,59.70,43.13,69223.33,491.24,248.46,1.00,47.09,409.40,349.61,178029.24,NaN
905,2024-03-20,10.23,10.48,9.57,10.67,252301.94,68032.40,42038.28,1941837.14,3711.36,624891.64,1774177.42,512.65,1338150.72,704063.82,10.30,10.09,11.67,10.30,8.93,59.21,55.58,47.00,71420.71,310.57,220.29,1.07,46.38,438.56,380.56,177122.23,NaN
906,2024-03-21,10.46,10.79,9.86,10.74,255683.78,67842.05,39532.67,1944665.76,3716.12,587764.66,1730681.23,510.24,1353276.31,699324.93,10.41,10.15,11.56,10.41,9.26,59.78,56.53,54.89,71633.30,292.76,200.90,0.99,45.02,445.95,411.29,167127.85,NaN
907,2024-03-22,10.63,10.99,10.01,10.85,266504.40,67787.31,42514.45,1971954.61,3727.83,548205.68,1693620.96,515.30,1337635.98,696277.88,10.48,10.22,11.56,10.48,9.40,60.74,57.05,58.08,74656.74,252.43,194.91,0.79,44.22,480.45,449.71,168761.65,NaN


In [22]:
ultimos_registros = feature_dataset.tail(5)
ultimos_registros.to_csv('h2o_models/predicciones.csv', index=False) 

Si predice maso menos bien para estas variables hacer lo siguiente:
- Evaluar los resultados por columna (definir alguna forma de hacer esto)

- Armar el clasificador (LSTM o CONV), entrenandolo sobre los datos del dataset original ( sin los ultimos 5 registros)
- Probar como predice el target (Tendencia) para los ultimos 5 registros con los datos generados por los modelos